In [1]:
import re
import pickle
import json
import pandas as pd

from library.common import Core
from pathlib import Path
cwd = Path.cwd()
prediction_horizon = 5
core = Core()
models = core.all_features

In [2]:
models_path = cwd.parent/'models'
models = list(models_path.glob('*.pkl'))

In [3]:
root_dir = cwd.parent
data_file= r'data/processed/params_forecast_params.json'
last_data_year_file = root_dir/data_file
file_path = last_data_year_file.as_posix()

In [4]:

with open(file_path, 'r') as datapoint:
    last_data_year = json.load(datapoint)

year_labels = [n for n in range(last_data_year.get('last_period') + 1,
                                last_data_year.get('last_period') + 1 +
                                prediction_horizon, 1)]
print('FORECAST LABELS: ', year_labels)

FORECAST LABELS:  [2021, 2022, 2023, 2024, 2025]


In [5]:
# regex = r"-([^-]\w*[^-])-([^-]\w*[^\.])"
regex = r"model-(Cluster [0-9])-([a-zA-Z]*_[a-zA-Z]*)"

forecasts = []
for model in models:
    # print('Processing:', model)
    fname = model.name
    matches = re.search(regex, fname)

    if matches:
        region = matches.group(1)
        feature = matches.group(2)

        print(region, feature)

        with open(model.as_posix(), 'rb') as pkl:
            optimal = pickle.load(pkl)

            f = optimal.predict(prediction_horizon)
            print('Forecast ', f)
            fc = zip(year_labels, f)
            forecast_df = pd.DataFrame(fc)
            forecast_df.columns = ['year', 'forecasts']
            forecasts.append([region, feature, forecast_df.to_numpy()])


Cluster 0 trade_openness
Forecast  [4399.42991989 4399.42991989 4399.42991989 4399.42991989 4399.42991989]
Cluster 0 renewable_energy
Forecast  [4904.56719227 4904.56719227 4904.56719227 4904.56719227 4904.56719227]
Cluster 1 renewable_energy
Forecast  [19.12396333 19.7277683  20.33157327 20.93537825 21.53918322]
Cluster 2 renewable_energy
Forecast  [1261.29904932 1299.80045405 1338.30185878 1376.80326352 1415.30466825]
Cluster 2 trade_openness
Forecast  [6307.6906624  6392.70829232 6477.72592224 6562.74355216 6647.76118209]
Cluster 1 trade_openness
Forecast  [58.72092823 59.54038443 60.35984064 61.17929684 61.99875305]


In [6]:
fc_path = cwd.parent/r'data/processed/ts_forecast.pkl'
with open(fc_path, 'wb') as pkl:
    pickle.dump(forecasts, pkl)

print("Done with Forecasts")

Done with Forecasts


In [10]:
from pathlib import Path
cwd = Path.cwd()
fc_path = cwd.parent/r'data/processed/ts_forecast.pkl'
with open(fc_path, 'rb') as fc_file:
    read_fc = pickle.load(fc_file)
print(read_fc)

[['Cluster 0', 'trade_openness', array([[2021.        , 4399.42991989],
       [2022.        , 4399.42991989],
       [2023.        , 4399.42991989],
       [2024.        , 4399.42991989],
       [2025.        , 4399.42991989]])], ['Cluster 0', 'renewable_energy', array([[2021.        , 4904.56719227],
       [2022.        , 4904.56719227],
       [2023.        , 4904.56719227],
       [2024.        , 4904.56719227],
       [2025.        , 4904.56719227]])], ['Cluster 1', 'renewable_energy', array([[2021.        ,   19.12396333],
       [2022.        ,   19.7277683 ],
       [2023.        ,   20.33157327],
       [2024.        ,   20.93537825],
       [2025.        ,   21.53918322]])], ['Cluster 2', 'renewable_energy', array([[2021.        , 1261.29904932],
       [2022.        , 1299.80045405],
       [2023.        , 1338.30185878],
       [2024.        , 1376.80326352],
       [2025.        , 1415.30466825]])], ['Cluster 2', 'trade_openness', array([[2021.        , 6307.6906624 ],
  